# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san mariano,7.5000,126.0000,30.08,67,91,1.67,PH,1722488001
1,1,thompson,55.7435,-97.8558,22.09,56,75,1.54,CA,1722487471
2,2,port-aux-francais,-49.3500,70.2167,3.97,84,100,13.28,TF,1722487932
3,3,la baneza,42.3003,-5.8977,20.20,88,0,2.08,ES,1722488004
4,4,bowen,-20.0167,148.2333,20.90,63,17,8.43,AU,1722488005


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size=10, color="Max Temp", 
                           hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"],
                           title="City Locations and Temperatures")
# Display the map
# YOUR CODE HERE


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Max Temp,City,Country,Humidity,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_city_data_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 30) & 
                                     (city_data_df["Humidity"] < 60)]
# Drop any rows with null values
# YOUR CODE HERE
filtered_city_data_df = filtered_city_data_df.dropna()
# Display sample data
# YOUR CODE HERE
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,thompson,55.7435,-97.8558,22.09,56,75,1.54,CA,1722487471
36,36,novyy urgal,51.0736,132.5642,29.68,43,93,3.31,RU,1722488044
43,43,pangody,65.8500,74.4859,23.68,49,38,1.95,RU,1722488053
65,65,ramnicu sarat,45.3833,27.0500,21.84,40,0,0.52,RO,1722488080
79,79,kirensk,57.7853,108.1119,20.06,44,100,3.41,RU,1722488097
...,...,...,...,...,...,...,...,...,...,...
536,536,samux,40.7647,46.4085,24.56,57,40,6.17,AZ,1722488663
539,539,rikaze,29.2500,88.8833,20.78,29,4,1.12,CN,1722488667
549,549,bulungu,-4.5500,18.6000,21.58,53,18,1.38,CD,1722488679
551,551,undurkhaan,47.3194,110.6556,24.38,54,61,5.87,MN,1722488681


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = "KBZX"
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,thompson,CA,55.7435,-97.8558,56,KBZX
36,novyy urgal,RU,51.0736,132.5642,43,KBZX
43,pangody,RU,65.8500,74.4859,49,KBZX
65,ramnicu sarat,RO,45.3833,27.0500,40,KBZX
79,kirensk,RU,57.7853,108.1119,44,KBZX


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 200
params = {
   "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
thompson - nearest hotel: Thompson Inn
novyy urgal - nearest hotel: No hotel found
pangody - nearest hotel: No hotel found
ramnicu sarat - nearest hotel: No hotel found
kirensk - nearest hotel: No hotel found
tafresh - nearest hotel: No hotel found
dzhangala - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
ridder - nearest hotel: No hotel found
berezovo - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
urengoy - nearest hotel: No hotel found
novo aripuana - nearest hotel: Hotel Novo Aripuanã
vernon - nearest hotel: No hotel found
xining - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
prineville - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
suzun - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
sosnovo-ozerskoye - nearest hotel: No hotel found
kingscliff - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
usinsk - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
1,thompson,CA,55.7435,-97.8558,56,Thompson Inn
36,novyy urgal,RU,51.0736,132.5642,43,No hotel found
43,pangody,RU,65.8500,74.4859,49,No hotel found
65,ramnicu sarat,RO,45.3833,27.0500,40,No hotel found
79,kirensk,RU,57.7853,108.1119,44,No hotel found
...,...,...,...,...,...,...
536,samux,AZ,40.7647,46.4085,57,No hotel found
539,rikaze,CN,29.2500,88.8833,29,No hotel found
549,bulungu,CD,-4.5500,18.6000,53,No hotel found
551,undurkhaan,MN,47.3194,110.6556,54,Khanburged Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", size=10, color="Humidity", 
    hover_cols=["City", "Country", "Hotel Name", "Humidity"], 
    title="Hotel Locations in Cities"
)
# Display the map
# YOUR CODE HERE
# hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)